# Parallelization: Distributes independent subtasks across multiple LLMs for concurrent processing

In [1]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

# Corrected way to retrieve the API key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

os.environ["GOOGLE_API_KEY"] = api_key

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [13]:
# This abstract representation a pool of threads which can be use to to execute callables/tasks in parallel.
from concurrent.futures import ThreadPoolExecutor # For parallel processing

# List of questions to ask the model
questions = [
    "What is the meaning of life?",
    "What is the capital of France?",
    "What is the capital of Germany?",
    "What is the capital of Italy?",
]

# Function to process a single thread
def process_question(question):
    """This function processes a single question and returns the answer."""
    return llm.invoke(f"Q: {question}")

# Create a thread pool with 5 threads
with ThreadPoolExecutor(max_workers=3) as executor:
    # Process all questions in parallel
    answers = list(executor.map(process_question, questions))

# Print the answers
for question, answer in zip(questions, answers):
    print(f"Q: {question}\nA: {answer.content}\n")

Q: What is the meaning of life?
A: There's no single, universally accepted answer to the question of the meaning of life.  It's a deeply personal and philosophical question that has been pondered by humans for millennia.  Different individuals and cultures find meaning in various things, including:

* **Relationships:**  Love, family, friendship, and community provide a sense of belonging and purpose for many.
* **Contribution:**  Making a difference in the world, whether through work, volunteering, or creative expression, can give life meaning.
* **Growth and learning:**  Continuously expanding one's knowledge, skills, and understanding can be a fulfilling pursuit.
* **Spiritual or religious beliefs:**  Faith and connection to something greater than oneself provide meaning and guidance for many.
* **Experiences:**  Travel, adventure, and pursuing passions can create rich and meaningful memories.
* **Creativity and self-expression:**  Expressing oneself through art, music, writing, or 

### Homework: Try to add context to the message and see if the results are better or not for questions

Note: This is just a demo to show how parallelization works. Python has some considerations for parallelization. (will be covered in future lectures)

## Routing: Dynamically select specialized LLM paths based on input characteristics

In [29]:
options =""" 'Life Insurance', 'Health Insurance', 'Car Insurance', 'Home Insurance', 'General Consultation' """
print(options.split(","))
input_text = "I want to know more about Life Insurance"

selection_prompt = f"""
user query: {input_text}
Analyze the user query and select the appropriate option from the following list: {options.split(",")}.
Fist explain your reasoning and then give me the answer in this XML format:

<reasoning>
Explain your reasoning here.
</reasoning>
<answer>
The answer goes here.
</answer>
"""

result = llm.invoke(selection_prompt)

[" 'Life Insurance'", " 'Health Insurance'", " 'Car Insurance'", " 'Home Insurance'", " 'General Consultation' "]


In [30]:
def extract_option(result):
    return result.content.split("<answer>")[1].split("</answer>")[0].strip()

def extract_reasoning(result):
    return result.content.split("<reasoning>")[1].split("</reasoning>")[0].strip()

option = extract_option(result)
reasoning = extract_reasoning(result)

print(f"Option: {option}")
print(f"Reasoning: {reasoning}")

Option: 'Life Insurance'
Reasoning: The user explicitly states "I want to know more about Life Insurance".  This directly indicates their interest in life insurance and not any other type of insurance or general consultation.  The query contains the keywords "Life Insurance", making it unambiguous.


In [40]:
option
print(type(option))
print(repr(option))

<class 'str'>
"'Life Insurance'"


In [48]:
# Based on the reasoning, print the tool free number( or any other information) for the selected option which is powered by business logic
# Remove extra single quotes if present
option = option.strip("'")

if option=='Life Insurance':
    print("Life Insurance Toll free number: 1800-123-4567")
elif option == "Health Insurance":
    print("Health Insurance Toll free number: 1800-234-5678")
elif option == "Car Insurance":
    print("Car Insurance Toll free number: 1800-345-6789")
elif option == "Home Insurance":
    print("Home Insurance Toll free number: 1800-456-7890")
else:
    print("General Consultation Toll free number: 1800-567-8901")

Life Insurance Toll free number: 1800-123-4567
